[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shhommychon/DeZero-Koki/blob/master/from_scratch_3/steps/step16.ipynb)

# 제1고지 미분 자동 계산

In [1]:
import numpy as np


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


In [2]:
# class Exp(Function):
#     def forward(self, x):
#         return np.exp(x)

#     def backward(self, gy):
#         x = self.input.data
#         gx = np.exp(x) * gy
#         return gx


# def exp(x):
#     return Exp()(x)


In [3]:
def numerical_diff(f, x, eps=1e-4):
    """

      param:
        f (Function): 미분의 대상이 되는 함수
        x (Variable): 미분을 계산하는 변수
        eps (float): 작은 값
    """
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

# 제2고지 자연스러운 코드로

## 11단계 가변 길이 인수(순전파 편)

## 12단계 가변 길이 인수(개선 편)

## 13단계 가변 길이 인수(역전파 편)

## 14단계 같은 변수 반복 사용

## 15단계 복잡한 계산 그래프(이론 편)

## 16단계 복잡한 계산 그래프(구현 편)

### 16.1 세대 추가

In [4]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0  # 세대 수를 기록하는 변수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대를 기록한다 (부모 세대 + 1)

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ output.grad for output in f.outputs ]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): gxs = gxs,

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)

    def cleargrad(self):
        self.grad = None


In [5]:
class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, *inputs):
        xs = [ x.data for x in inputs ]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple): ys = ys,
        outputs = [ Variable(as_array(y)) for y in ys ]

        self.generation = max([x.generation for x in inputs])  # 입력 변수 중 가장 큰 generation의 수를 선택
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 16.2 세대 순으로 꺼내기

In [6]:
class DummyFunction: pass

generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
    f = DummyFunction()  # 더미 함수 클래스
    f.generation = g
    funcs.append(f)

[f.generation for f in funcs]

[2, 0, 1, 4, 2]

In [7]:
funcs.sort(key=lambda x: x.generation)  # 리스트 정렬
[f.generation for f in funcs]

[0, 1, 2, 2, 4]

In [8]:
f = funcs.pop()
f.generation

4

### 16.3 Variable 클래스의 backward

In [9]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0  # 세대 수를 기록하는 변수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대를 기록한다 (부모 세대 + 1)

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        #########################
        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)
        #########################

        while funcs:
            f = funcs.pop()
            gys = [ output.grad for output in f.outputs ]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): gxs = gxs,

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    #########################
                    add_func(x.creator)
                    #########################

    def cleargrad(self):
        self.grad = None


### 16.4 동작 확인

In [10]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


In [11]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


In [12]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0
